In [ ]:
import cv2
import math
from ultralytics import YOLO
import os

def create_video_writer(video_cap, output_filename):
    frame_width = int(video_cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(video_cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(video_cap.get(cv2.CAP_PROP_FPS))

    fourcc = cv2.VideoWriter_fourcc(*'MP4V')
    writer = cv2.VideoWriter(output_filename, fourcc, fps, (frame_width, frame_height))

    return writer

def create_directory_if_not_exists(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)

def save_frame(frame, output_directory, frame_number):
    cv2.imwrite(os.path.join(output_directory, f"frame_{frame_number}.png"), frame)

def main():
    cap = cv2.VideoCapture(0)
    writer = create_video_writer(cap, "ConstructionSiteSafetyOutput.mp4")
    model = YOLO("best.pt")
    classNames = ['Excavator', 'Gloves', 'Hardhat', 'Ladder', 'Mask', 'NO-Hardhat', 'NO-Mask', 'NO-Safety Vest',
                  'Person', 'SUV', 'Safety Cone', 'Safety Vest', 'bus', 'dump truck', 'fire hydrant', 'machinery',
                  'mini-van', 'sedan', 'semi', 'trailer', 'truck and trailer', 'truck', 'van', 'vehicle', 'wheel loader']

    output_directories = {
        'no_hardhat': 'output_no_hardhat',
        'no_gloves': 'output_no_gloves',
        'no_safety_vest': 'output_no_safety_vest'
    }

    for directory in output_directories.values():
        create_directory_if_not_exists(directory)

    frame_number = 0
    while True:
        success, img = cap.read()
        if not success:
            break

        results = model(img, stream=True)

        violation_detected = False

        for r in results:
            boxes = r.boxes
            for box in boxes:
                x1, y1, x2, y2 = box.xyxy[0]
                x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
                w, h = x2 - x1, y2 - y1

                conf = math.ceil((box.conf[0] * 100)) / 100
                cls = int(box.cls[0])
                currentClass = classNames[cls]
                print(currentClass)

                if conf > 0.5:
                    if currentClass == 'NO-Hardhat':
                        save_frame(img, output_directories['no_hardhat'], frame_number)
                        violation_detected = True
                    elif currentClass == 'NO-Gloves':
                        save_frame(img, output_directories['no_gloves'], frame_number)
                        violation_detected = True
                    elif currentClass == 'NO-Safety Vest':
                        save_frame(img, output_directories['no_safety_vest'], frame_number)
                        violation_detected = True

                    if currentClass == 'NO-Hardhat' or currentClass == 'NO-Safety Vest' or currentClass == "NO-Mask":
                        myColor = (0, 0, 255)
                    elif currentClass == 'Hardhat' or currentClass == 'Safety Vest' or currentClass == "Mask":
                        myColor = (0, 255, 0)
                    else:
                        myColor = (255, 0, 0)

                    image = cv2.putText(img, "{}".format(currentClass), (x1, y1), cv2.FONT_HERSHEY_SIMPLEX,
                                        1, (255, 0, 0), 2, cv2.LINE_AA)
                    cv2.rectangle(img, (x1, y1), (x2, y2), myColor, 3)

        cv2.imshow("STM-Detection-Project", img)
        writer.write(img)
        frame_number += 1

        key = cv2.waitKey(1)
        if key == ord('q'):
            break

    cap.release()
    writer.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()
